In [21]:
import pandas
import altair as alt
import pandas.io.json
import swifter
# import dask.dataframe
# from dask.distributed import Client

# https://github.com/pandas-dev/pandas/issues/26068#issuecomment-550900208
import json
pandas.io.json._json.loads = lambda s, *a, **kw: json.loads(s)

# dask.config.set({'temporary_directory': '/tmp'})
alt.data_transformers.enable('data_server')
pandas.set_option('max_colwidth', 200)

# client = Client()
# client

In [9]:
# data = dask.dataframe.read_json(
#     'skimage.jsonl',
#     blocksize=1000 * 1024,
#     meta=pandas.DataFrame.from_dict({"function": ["str"], "params": [{}]})
# )
# data
data = pandas.read_json('data/raw/skimage.jsonl', lines=True)

In [10]:
data.info()
data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 513734 entries, 0 to 513733
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   location  513734 non-null  object
 1   function  513734 non-null  object
 2   params    513734 non-null  object
dtypes: object(3)
memory usage: 11.8+ MB


,location,function,params
0,/Users/saul/Downloads/scikit-image-0.17.0/skimage/_shared/tests/test_fast_exp.py:7,builtins.getattr,"{'0': {'t': 'builtins.module', 'v': 'numpy'}, '1': 'linspace'}"
1,/Users/saul/Downloads/scikit-image-0.17.0/skimage/_shared/tests/test_fast_exp.py:7,numpy.linspace,"{'start': -5, 'stop': 0, 'num': 5000, 'endpoint': {'t': 'builtins.bool', 'v': True}}"
2,/Users/saul/Downloads/scikit-image-0.17.0/skimage/_shared/tests/test_fast_exp.py:10,numpy.exp,"{'0': {'t': 'numpy.ndarray', 'v': [[5000], 'float64']}}"
3,/Users/saul/Downloads/scikit-image-0.17.0/skimage/_shared/tests/test_fast_exp.py:13,builtins.iter,"{'0': {'t': 'numpy.ndarray', 'v': [[5000], 'float64']}}"
4,/Users/saul/Downloads/scikit-image-0.17.0/skimage/_shared/tests/test_fast_exp.py:13,numpy.array,"{'0': [0.0067521147429943085, 0.006757751107215881, 0.006763387471437454, 0.006769023835659027, 0.0067746601998806, 0.006780296564102173, 0.006785932928323746, 0.006791569292545319, 0.006797205656..."
...,...,...,...
513729,/Users/saul/Downloads/scikit-image-0.17.0/skimage/_shared/utils.py:238,_operator.gt,"{'a': {'t': 'numpy.ndarray', 'v': [[2], 'int64']}, 'b': 0.5}"
513730,/Users/saul/Downloads/scikit-image-0.17.0/skimage/_shared/utils.py:238,_operator.setitem,"{'a': {'t': 'numpy.ndarray', 'v': [[2], 'int64']}, 'b': {'t': 'numpy.ndarray', 'v': [[2], 'bool']}, 'c': {'t': 'numpy.ndarray', 'v': [[0], 'int64']}}"
513731,/Users/saul/Downloads/scikit-image-0.17.0/skimage/_shared/utils.py:241,builtins.getattr,"{'0': {'t': 'builtins.module', 'v': 'numpy'}, '1': 'testing'}"
513732,/Users/saul/Downloads/scikit-image-0.17.0/skimage/_shared/utils.py:241,builtins.getattr,"{'0': {'t': 'builtins.module', 'v': 'numpy.testing'}, '1': 'assert_allclose'}"


In [14]:
just_numpy = data[data.function.str.startswith("numpy.")]
counts = just_numpy.groupby('function').count()
counts['occurance'] = counts['params'] / counts['params'].sum()

In [15]:
alt.Chart(counts.reset_index()).mark_bar().encode(
    y=alt.Y('function:N', sort='-x'),
    x=alt.X('occurance:Q', axis=alt.Axis(format='%'), scale=alt.Scale(type='log')),
    tooltip=['function', 'occurance', 'params']
)

alt.Chart(...)

In [24]:
numpy_private = just_numpy[just_numpy.function.str.contains("._", regex=False)]


In [25]:
pandas.options.display.max_rows = 999

numpy_private.groupby(['function', 'location']).count()

,,params
function,location,
numpy.core._dtype._construction_repr,/usr/local/Caskroom/miniconda/base/envs/python-record-api/lib/python3.8/site-packages/numpy/core/_dtype.py:58,3
numpy.core._dtype._kind_name,/usr/local/Caskroom/miniconda/base/envs/python-record-api/lib/python3.8/site-packages/numpy/core/_dtype.py:344,51
numpy.core._dtype._name_includes_bit_suffix,/usr/local/Caskroom/miniconda/base/envs/python-record-api/lib/python3.8/site-packages/numpy/core/_dtype.py:347,51
numpy.core._multiarray_umath.implement_array_function,<__array_function__ internals>:5,2971
numpy.core.fromnumeric._all_dispatcher,<__array_function__ internals>:4,9
numpy.core.fromnumeric._amax_dispatcher,<__array_function__ internals>:4,46
numpy.core.fromnumeric._amin_dispatcher,<__array_function__ internals>:4,47
numpy.core.fromnumeric._prod_dispatcher,<__array_function__ internals>:4,854
numpy.core.fromnumeric._shape_dispatcher,<__array_function__ internals>:4,853


In [26]:
getattr = get_args('builtins.getattr')

/usr/local/Caskroom/miniconda/base/envs/python-record-api/lib/python3.8/site-packages/tqdm/std.py:666: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [23]:
getattr

,0,1
function,,
builtins.getattr,builtins.module,'byte'
builtins.getattr,builtins.module,'ubyte'
builtins.getattr,builtins.module,'short'
builtins.getattr,builtins.module,'ushort'
builtins.getattr,builtins.module,'intc'
...,...,...
builtins.getattr,builtins.module,'uint8'
builtins.getattr,numpy.ndarray,'ndim'
builtins.getattr,numpy.ndarray,'ndim'


In [24]:
alt.Chart(getitem.groupby('b').count().reset_index()).mark_bar().encode(
    y=alt.Y('b:N', sort='-x'),
    x=alt.X('a:Q', scale=alt.Scale(type='log')),
    tooltip=['b', 'a']
)

alt.Chart(...)

In [24]:
def stringify(v):
    """
    Turn a complex type into a simpler string, yes with less data, but easier to group by.
    """
    if isinstance(v, int):
        return "int"
    if isinstance(v, str):
        return str(repr(v))
    if isinstance(v, float):
        return "float"
    if isinstance(v, list):
        return f"List[{stringify(v[0]) if v else 'Any'}]"
    if v is None:
        return 'None'
    t = v['t']
    if t == 'builtins.type':
        return f"Type[{v['v']}]"
    if t == 'builtins.module':
        return v['v']
    if t == 'tuple':
        return f"Tuple[{', '.join(stringify(a) for a in v['v'])}]"
    return t
        

In [25]:
def get_args(fn_name):
    return data[data.index == fn_name].swifter.apply(
        lambda x: pandas.Series({k: stringify(v) for k, v in x['params'].items()}),
        axis=1
    )

In [10]:
def make_arg_chart(df):
    selections = [alt.selection_multi(fields=[col]) for col in df.columns]
    charts = []
    for i, col in enumerate(df.columns):
        new_chart = alt.Chart(df).mark_bar().encode(
            alt.Y(col, sort='-x', type='nominal'),
            alt.X('count()', scale=alt.Scale(type='log')),
            color=alt.condition(selections[i], alt.value('lightblue'), alt.value('lightgray'))
        ).add_selection(
            selections[i]
        )
        for other_i, selection in enumerate(selections):
            # don't filter on my own selection
            if other_i == i:
                continue
            new_chart = new_chart.transform_filter(selection)
        charts.append(new_chart)
    return alt.concat(*charts, columns=3)

In [11]:
def inspect_functionn(fn_name):
    df = get_args(fn_name)
    return make_arg_chart(df)